#### pip dependencies
- pymongo

#### other dependencies
- https://github.com/veryhappythings/dotamatch (modified version)

## Get latest matches from steam API

In [54]:
import pprint
import json, jsonpickle
from dotamatch import get_key
from dotamatch.history import MatchHistoryBySequenceNum
from dotamatch.players import PlayerSummaries
from dotamatch.heroes import Heroes

key = get_key()
history = MatchHistoryBySequenceNum(key)

In [55]:
#1100000000 come from bruce forcing match_seq_num in api
for match in history.matches(start_at_match_seq_num = 1100000000):
    latest = match

In [63]:
latest.match_seq_num
#data = json.loads(latest)

1100000202

## Connect to MongoDB

In [38]:
from pymongo import MongoClient, ASCENDING
client = MongoClient('localhost', 3001)

In [39]:
db = client.meteor
matches = db.matches

In [23]:
matches.create_index([("match_id", ASCENDING)], unique=True)

u'match_id_1'

In [11]:
ldict = latest.__dict__

In [12]:
ldict.pop("parent", None)

In [13]:
matches.insert_one(ldict)

   ## Create / Update Sequence Data

In [40]:
preferences = db.preferences

In [46]:
# only run this first time
preferences.insert_one({'type': "settings", 'match_sequence_num': 1100000000})

In [51]:
preferences.find_one()["match_sequence_num"]

123

In [52]:
preferences.update_one({"type": "settings"}, {"$set": {"match_sequence_num": 1100000000}})